In [ ]:
#!/usr/bin/env python3
import os
import time
import tempfile
import mimetypes
import subprocess
import gradio as gr
from google import genai
from google.genai import types

# -----------------------------
# Config & Globals
# -----------------------------
API_KEY = ""
CLIENT = None
CURRENT_STORE = None
FALLBACK_FILE = "/mnt/data/sample.png"

# -----------------------------
# Helpers
# -----------------------------
def _ensure_client(key):
    global CLIENT
    if CLIENT:
        return CLIENT
    CLIENT = genai.Client(api_key=key)
    return CLIENT

def _save_temp(path):
    fd, tmp = tempfile.mkstemp(suffix=os.path.splitext(path)[1])
    os.close(fd)
    with open(path, "rb") as f_in, open(tmp, "wb") as f_out:
        f_out.write(f_in.read())
    return tmp

# -----------------------------
# MULTIPLE FILE UPLOAD + IMPORT
# -----------------------------
def process_files(files):
    global CLIENT, CURRENT_STORE

    if CLIENT is None:
        return "⚠️ Please connect API first."

    if not files or len(files) == 0:
        return "❌ No files uploaded."

    try:
        # Create store once
        if CURRENT_STORE is None:
            CURRENT_STORE = CLIENT.file_search_stores.create(
                config={"display_name": f"store_{int(time.time())}"}
            )

        uploaded_count = 0

        for f in files:
            src = f.name
            if not os.path.exists(src):
                continue

            tmp = _save_temp(src)

            op = CLIENT.file_search_stores.upload_to_file_search_store(
                file=tmp,
                file_search_store_name=CURRENT_STORE.name,
                config={"display_name": os.path.basename(src)}
            )

            # Poll until done
            while not op.done:
                time.sleep(1)
                op = CLIENT.operations.get(op)

            uploaded_count += 1
            os.remove(tmp)

        return f"✅ Imported {uploaded_count} file(s) into store: {CURRENT_STORE.name}"

    except Exception as e:
        return f"❌ Error uploading: {e}"


# -----------------------------
# CHAT (ONLY ANSWER FROM FILES)
# -----------------------------
def chat_logic(message, history):
    global CLIENT, CURRENT_STORE

    if CLIENT is None:
        return "⚠️ Connect API first."

    if CURRENT_STORE is None:
        return "⚠️ Upload documents first."

    # File Search tool
    tool = types.Tool(
        file_search=types.FileSearch(
            file_search_store_names=[CURRENT_STORE.name]
        )
    )

    # STRICT system rule
    system_instruction = (
        "You MUST answer ONLY using information found inside the uploaded documents. "
        "If the information is not present in the uploaded files, respond exactly with: "
        "'I cannot find this information in the uploaded files.' "
        "Do NOT use outside knowledge. Do NOT guess."
    )

    config = types.GenerateContentConfig(
        tools=[tool],
        system_instruction=system_instruction,
        temperature=0.0,
        max_output_tokens=800,
    )

    response = CLIENT.models.generate_content(
        model="gemini-2.5-flash",
        contents=[message],
        config=config
    )

    return response.text


# -----------------------------
# Gradio UI
# -----------------------------
def setup_client(api_key_input):
    key = (api_key_input or "").strip() or API_KEY
    if not key:
        return "⚠️ Please enter your Gemini API Key"
    try:
        _ensure_client(key)
        return "✅ Gemini client connected."
    except Exception as e:
        return f"❌ Error: {e}"


with gr.Blocks() as app:
    gr.Markdown("## Gemini File Search Assistant (Multi-Document + Strict Answering)")

    with gr.Row():
        with gr.Column(scale=1):
            api_input = gr.Textbox(label="Gemini API Key", type="password")
            btn = gr.Button("Connect")
            status = gr.Markdown("Not connected")

            file_input = gr.File(
                label="Upload many documents",
                file_count="multiple"
            )
            upload_status = gr.Markdown("")

        with gr.Column(scale=2):
            chat = gr.ChatInterface(
                fn=chat_logic,
                type="messages",
                title="Document-Based Answering Only"
            )

    btn.click(setup_client, inputs=api_input, outputs=status)
    file_input.change(process_files, inputs=file_input, outputs=upload_status)

if __name__ == "__main__":
    app.launch()
